# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

```python
{
   "Organ Failure History" : [
        { "matches": "Nonoperative", "points": 5 },
        { "matches": "Emergency", "points": 5 },
        { "matches": "Elective", "points": 2 },
        { "matches": "None", "points": 0 }
    ],
    "Age": [
        { "min": 0, "max": 45, "points": 0 },
        { "min": 45, "max": 55, "points": 2 },
        { "min": 55, "max": 65, "points": 3 },
        { "min": 65, "max": 75, "points": 5 },
        { "min": 75, "max": 999, "points": 6 }
    ],
    "Creatinine": [
        { "rules": [ { "prop": "Acute Renal Failure", "matches": "Acute Renal Failure" }, { "min": 3.5, "max": 999 } ], "points": 8 },
        { "rules": [ { "prop": "Acute Renal Failure", "matches": "Acute Renal Failure" }, { "min": 2.0, "max": 3.5 } ], "points": 6 },
        { "rules": [ { "prop": "Acute Renal Failure", "matches": "Chronic Renal Failure", "q": true }, { "min": 3.5, "max": 999 } ], "points": 4 },
        { "rules": [ { "prop": "Acute Renal Failure", "matches": "Acute Renal Failure" }, { "min": 1.5, "max": 2.0 } ], "points": 4 },
        { "rules": [ { "prop": "Acute Renal Failure", "matches": "Chronic Renal Failure" }, { "min": 2.0, "max": 3.5 } ], "points": 3 },
        { "rules": [ { "prop": "Acute Renal Failure", "matches": "Chronic Renal Failure" }, { "min": 1.5, "max": 2.0 } ], "points": 2 },
        { "min": 0.6, "max": 1.5, "points": 0 },
        { "min": 0, "max": 0.6, "points": 2 }
    ],
    "Temperature": [
        { "min": 0, "max": 30, "points": 4 },
        { "min": 30, "max": 32, "points": 3 },
        { "min": 32, "max": 34, "points": 2 },
        { "min": 34, "max": 36, "points": 1 },
        { "min": 36, "max": 38.5, "points": 0 },
        { "min": 38.5, "max": 39, "points": 1 },
        { "min": 39, "max": 41, "points": 3 },
        { "min": 41, "max": 100, "points": 4 }
    ],
    "Heart Rate": [
        { "min": 0, "max": 40, "points": 4 },
        { "min": 40, "max": 55, "points": 3 },
        { "min": 55, "max": 70, "points": 2 },
        { "min": 70, "max": 110, "points": 0 },
        { "min": 110, "max": 140, "points": 2 },
        { "min": 140, "max": 180, "points": 3 },
        { "min": 180, "max": 999, "points": 4 }
    ],
    "Respiratory Rate": [
        { "min": 0, "max": 6, "points": 4 },
        { "min": 6, "max": 10, "points": 2 },
        { "min": 10, "max": 12, "points": 1 },
        { "min": 12, "max": 25, "points": 0 },
        { "min": 25, "max": 35, "points": 1 },
        { "min": 35, "max": 50, "points": 3 },
        { "min": 50, "max": 999, "points": 4 }
    ],
    "pH": [
        { "min": 0, "max": 7.15, "points": 4 },
        { "min": 7.15, "max": 7.25, "points": 3 },
        { "min": 7.25, "max": 7.33, "points": 2 },
        { "min": 7.33, "max": 7.50, "points": 0 },
        { "min": 7.50, "max": 7.60, "points": 1 },
        { "min": 7.60, "max": 7.70, "points": 3 },
        { "min": 7.70, "max": 999, "points": 4 }
    ],
    "Sodium": [
        { "min": 0, "max": 111, "points": 4 },
        { "min": 111, "max": 120, "points": 3 },
        { "min": 120, "max": 130, "points": 2 },
        { "min": 130, "max": 150, "points": 0 },
        { "min": 150, "max": 155, "points": 1 },
        { "min": 155, "max": 160, "points": 2 },
        { "min": 160, "max": 180, "points": 3 },
        { "min": 180, "max": 999, "points": 4 }
    ],
    "Potassium": [
        { "min": 0, "max": 2.5, "points": 4 },
        { "min": 2.5, "max": 3.0, "points": 2 },
        { "min": 3.0, "max": 3.5, "points": 1 },
        { "min": 3.5, "max": 5.5, "points": 0 },
        { "min": 5.5, "max": 6.0, "points": 1 },
        { "min": 6.0, "max": 7.0, "points": 3 },
        { "min": 7.0, "max": 999, "points": 4 }
    ],
    "Hematocrit" : [
        { "min": 0, "max": 20, "points": 4 },
        { "min": 20, "max": 30, "points": 2 },
        { "min": 30, "max": 46, "points": 0 },
        { "min": 46, "max": 50, "points": 1 },
        { "min": 50, "max": 60, "points": 2 },
        { "min": 60, "max": 999, "points": 4 }
    ],
    "White Blood Count": [
        { "min": 0, "max": 1, "points": 4 },
        { "min": 1, "max": 3, "points": 2 },
        { "min": 3, "max": 15, "points": 0 },
        { "min": 15, "max": 20, "points": 1 },
        { "min": 20, "max": 40, "points": 2 },
        { "min": 40, "max": 999, "points": 4 }
    ],
    "FiO2": [
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 0, "max": 55 } ], "points": 4 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 55, "max": 61 } ], "points": 3 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 61, "max": 71 } ], "points": 1 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 71, "max": 999 } ], "points": 0 },
        
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 0, "max": 200 } ], "points": 0},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 200, "max": 350 } ], "points": 2},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 350, "max": 500 } ], "points": 3},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 500, "max": 999 } ], "points": 4}
    ],
    "Glasgow Coma Scale": [
        {"min": 1, "max": 16, "points": "15-_"}
    ]
}
```

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [62]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())

In [63]:
('Organ Failure History' in rules.keys())

True

In [64]:
('Age' in rules.keys())

True

In [65]:
('Temperature' in rules.keys())

True

In [66]:
('pH' in rules.keys())

True

In [67]:
('Heart Rate' in rules.keys())

True

In [68]:
('Respiratory Rate' in rules.keys())

True

In [69]:
('Sodium' in rules.keys())

True

In [70]:
('Potassium' in rules.keys())

True

In [71]:
('Creatinine' in rules.keys())

True

In [72]:
('Hematocrit' in rules.keys())

True

In [73]:
('White Blood Count' in rules.keys())

True

In [74]:
('FiO2' in rules.keys())

True

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [75]:
def evaluate_rule(config_file, rule_name, data):
    """
        functions, only one function can evaluate all the rules
        this function supports the following syntax -

            "rule name": [
                // all rules here
            ]

        Only one of the rules in the list is matched.
        Each rule in the list is either a simple rule, or a complex rule.


        simple rule - a simple rule only has 1 condition
        current property in considerations is the "rule name", by default
            eg 1 -     { "min": 10, "max": 15, "points": 5 }
                (eg 1) if the current property in consideration is in range 10-14 (inclusive), 5 points will be added
            
            eg 2 -     { "matches": 10, "points": 2 }
                (eg 2) if the current property in consideration matches 2, points will be added
            
            eg 3 -     { "matches": "ab", "points": 1 }
                (eg 3) if the current property in consideration matches "ab", 1 point will be added
            
            eg 4 -     { "prop": "Sodium", "matches": 3, "points": 5 }
                (eg 4) this rule first changes the current property in consideration
                    to "Sodium", If the patient's "Sodium" level is 3, add 5 points
            
            eg 5 -     { "prop": "GCS" "min": 1, "max": 15, "points": "15-_" }
                (eg 5) this checks if current property in consideration is in range 1-14 (inclusive). If yes, it just
                        replaces all '_' in "15-_", with the value of the property and evaluates. Suppose
                        value of property "GCS" is 10, the "15-_" will become "15-10". Now the equation
                        will be solved and points will be added accordingly, here 15-10 = 5, so 5 points will
                        be added


        complex rule - a complex rule can have many conditions, each of which has to be satisfied as a whole
                        a complex rule is basically made up of many simple rules
            eg 6 -     { "rules": [ { "min": 10, "max": 15 } ], "points": 5 }
                (eg 6) this checks if the value of current property in consideration is in
                        the range 10-14(inclusive), if yes, it adds 5 points
            
            eg 7 -     { "rules": [ { "min": 10, "max": 15 }, { "matches": 12 } ], "points": 2 }
                (eg 7) this 2 conditions, first it checks if the value of current property in consideration is in
                        the range 10-14(inclusive), if yes, it moves on to the next condition and checks
                        if value matches 12, if yes, it adds 2 points
            
            eg 8 -     { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 0, "max": 55 } ], "points": 4 }
                (eg 8) this has 2 conditions, if current property in consideration is in range 0-49(inclusive)
                and "PaO2" is in range 0-54(inclusive), then 4 points will be added

    """
    rules = config_file[rule_name]
    points = 0

    for rule in rules:
        if "rules" in rule:
            # this rule contains sub rules
            sub_rules = rule[ "rules" ]
        else:
            sub_rules = [ rule ]
        current_value, current_points, sub_rule_unmatched = data[rule_name], rule["points"], False

        for sub_rule in sub_rules:
            if "prop" in sub_rule:
                # if this rule uses property other than the current rule name,
                # set its value to `value`
                prop = sub_rule["prop"]
                if prop in data:
                    value = data[prop]
                else:
                    # all rules were not matched
                    sub_rule_unmatched = True
                    break
            else:
                value = current_value
            if "matches" in sub_rule:
                # exact match failed
                if value != sub_rule["matches"]:
                    # all rules were not matched
                    sub_rule_unmatched = True
                    break
            elif "min" in sub_rule and "max" in sub_rule:
                # range match failed
                if value < sub_rule["min"] or value >= sub_rule["max"]:
                    # all rules were not matched
                    sub_rule_unmatched = True
                    break
            
            if isinstance (current_points, ("".__class__, u"".__class__)):
                # if points is string
                # replace _ with `value` and evaluate the mathematical expression
                points = eval(current_points.replace("_", repr(value)))
            else:
                points = current_points

        if not sub_rule_unmatched:
            break
    return points


### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [76]:
evaluate_rule(rules, "Organ Failure History", { "Organ Failure History": "Nonoperative" })

5

In [77]:
evaluate_rule(rules, "Organ Failure History", { "Organ Failure History": "Elective" })

2

In [78]:
evaluate_rule(rules, "Organ Failure History", { "Organ Failure History": "None" })

0

In [79]:
evaluate_rule(rules, "Age", {"Age": 40})

0

In [80]:
evaluate_rule(rules, "Age", {"Age": 50})

2

In [81]:
evaluate_rule(rules, "Age", {"Age": 60})

3

In [82]:
evaluate_rule(rules, "Creatinine", { "Creatinine": 0.5 })

2

In [83]:
evaluate_rule(rules, "Creatinine", { "Creatinine": 2.1, "Acute Renal Failure": "Acute Renal Failure" })

6

In [84]:
evaluate_rule(rules, "Creatinine", { "Creatinine": 3.6, "Acute Renal Failure": "Chronic Renal Failure" })

4

In [85]:
evaluate_rule(rules, "Temperature", {"Temperature": 33})

2

In [86]:
evaluate_rule(rules, "Temperature", {"Temperature": 37.5})

0

In [87]:
evaluate_rule(rules, "Temperature", {"Temperature": 60})

4

In [88]:
evaluate_rule(rules, "Heart Rate", {"Heart Rate": 35})

4

In [89]:
evaluate_rule(rules, "Heart Rate", {"Heart Rate": 160})

3

In [90]:
evaluate_rule(rules, "Heart Rate", {"Heart Rate": 100})

0

In [91]:
evaluate_rule(rules, "Respiratory Rate", {"Respiratory Rate": 11})

1

In [92]:
evaluate_rule(rules, "Respiratory Rate", {"Respiratory Rate": 6})

2

In [93]:
evaluate_rule(rules, "Respiratory Rate", {"Respiratory Rate": 1})

4

In [94]:
evaluate_rule(rules, "pH", {"pH": 7.14})

4

In [95]:
evaluate_rule(rules, "pH", {"pH": 7.33})

0

In [96]:
evaluate_rule(rules, "pH", {"pH": 7.28})

2

In [97]:
evaluate_rule(rules, "Sodium", {"Sodium": 125})

2

In [98]:
evaluate_rule(rules, "Sodium", {"Sodium": 140})

0

In [99]:
evaluate_rule(rules, "Sodium", {"Sodium": 190})

4

In [100]:
evaluate_rule(rules, "Potassium", {"Potassium": 2.6})

2

In [101]:
evaluate_rule(rules, "Potassium", {"Potassium": 5.4})

0

In [102]:
evaluate_rule(rules, "Potassium", {"Potassium": 7.9})

4

In [103]:
evaluate_rule(rules, "Hematocrit", {"Hematocrit": 47})

1

In [104]:
evaluate_rule(rules, "Hematocrit", {"Hematocrit": 29})

2

In [105]:
evaluate_rule(rules, "Hematocrit", {"Hematocrit": 70})

4

In [106]:
evaluate_rule(rules, "White Blood Count", {"White Blood Count": 10})

0

In [107]:
evaluate_rule(rules, "White Blood Count", {"White Blood Count": 2})

2

In [108]:
evaluate_rule(rules, "White Blood Count", {"White Blood Count": 50})

4

In [109]:
evaluate_rule(rules, "FiO2", {"FiO2": 40, "PaO2": 55, "A-a Gradient": 250})

3

In [110]:
evaluate_rule(rules, "FiO2", {"FiO2": 45, "PaO2": 61, "A-a Gradient": 350})

1

In [111]:
evaluate_rule(rules, "FiO2", {"FiO2": 60, "PaO2": 55, "A-a Gradient": 199})

0

In [112]:
evaluate_rule(rules, "FiO2", {"FiO2": 65, "PaO2": 55, "A-a Gradient": 300})

2

In [113]:
evaluate_rule(rules, "Glasgow Coma Scale", {"Glasgow Coma Scale": 7})

8

In [114]:
evaluate_rule(rules, "Glasgow Coma Scale", {"Glasgow Coma Scale": 10})

5

In [115]:
evaluate_rule(rules, "Glasgow Coma Scale", {"Glasgow Coma Scale": 2})

13

In [116]:
assert(evaluate_rule(rules, "Organ Failure History", { "Organ Failure History": "Nonoperative" }) == 5)
assert(evaluate_rule(rules, "Organ Failure History", { "Organ Failure History": "Elective" }) == 2)
assert(evaluate_rule(rules, "Organ Failure History", { "Organ Failure History": "None" }) == 0)

assert(evaluate_rule(rules, "Age", {"Age": 40}) == 0)
assert(evaluate_rule(rules, "Age", {"Age": 50}) == 2)
assert(evaluate_rule(rules, "Age", {"Age": 60}) == 3)

assert(evaluate_rule(rules, "Creatinine", { "Creatinine": 0.5 }) == 2)
assert(evaluate_rule(rules, "Creatinine", { "Creatinine": 2.1, "Acute Renal Failure": "Acute Renal Failure" }) == 6)
assert(evaluate_rule(rules, "Creatinine", { "Creatinine": 3.6, "Acute Renal Failure": "Chronic Renal Failure" }) == 4)

assert(evaluate_rule(rules, "Temperature", {"Temperature": 33}) == 2)
assert(evaluate_rule(rules, "Temperature", {"Temperature": 37.5}) == 0)
assert(evaluate_rule(rules, "Temperature", {"Temperature": 60}) == 4)

assert(evaluate_rule(rules, "Heart Rate", {"Heart Rate": 35}) == 4)
assert(evaluate_rule(rules, "Heart Rate", {"Heart Rate": 160}) == 3)
assert(evaluate_rule(rules, "Heart Rate", {"Heart Rate": 100}) == 0)

assert(evaluate_rule(rules, "Respiratory Rate", {"Respiratory Rate": 11}) == 1)
assert(evaluate_rule(rules, "Respiratory Rate", {"Respiratory Rate": 6}) == 2)
assert(evaluate_rule(rules, "Respiratory Rate", {"Respiratory Rate": 1}) == 4)

assert(evaluate_rule(rules, "pH", {"pH": 7.14}) == 4)
assert(evaluate_rule(rules, "pH", {"pH": 7.33}) == 0)
assert(evaluate_rule(rules, "pH", {"pH": 7.28}) == 2)

assert(evaluate_rule(rules, "Sodium", {"Sodium": 125}) == 2)
assert(evaluate_rule(rules, "Sodium", {"Sodium": 140}) == 0)
assert(evaluate_rule(rules, "Sodium", {"Sodium": 190}) == 4)

assert(evaluate_rule(rules, "Potassium", {"Potassium": 2.6}) == 2)
assert(evaluate_rule(rules, "Potassium", {"Potassium": 5.4}) == 0)
assert(evaluate_rule(rules, "Potassium", {"Potassium": 7.9}) == 4)

assert(evaluate_rule(rules, "Hematocrit", {"Hematocrit": 47}) == 1)
assert(evaluate_rule(rules, "Hematocrit", {"Hematocrit": 29}) == 2)
assert(evaluate_rule(rules, "Hematocrit", {"Hematocrit": 70}) == 4)

assert(evaluate_rule(rules, "White Blood Count", {"White Blood Count": 10}) == 0)
assert(evaluate_rule(rules, "White Blood Count", {"White Blood Count": 2}) == 2)
assert(evaluate_rule(rules, "White Blood Count", {"White Blood Count": 50}) == 4)

assert(evaluate_rule(rules, "FiO2", {"FiO2": 40, "PaO2": 55, "A-a Gradient": 250}) == 3)
assert(evaluate_rule(rules, "FiO2", {"FiO2": 45, "PaO2": 61, "A-a Gradient": 350}) == 1)
assert(evaluate_rule(rules, "FiO2", {"FiO2": 60, "PaO2": 55, "A-a Gradient": 199}) == 0)
assert(evaluate_rule(rules, "FiO2", {"FiO2": 65, "PaO2": 55, "A-a Gradient": 300}) == 2)

assert(evaluate_rule(rules, "Glasgow Coma Scale", {"Glasgow Coma Scale": 7}) == 8)
assert(evaluate_rule(rules, "Glasgow Coma Scale", {"Glasgow Coma Scale": 10}) == 5)
assert(evaluate_rule(rules, "Glasgow Coma Scale", {"Glasgow Coma Scale": 2}) == 13)

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [117]:
def apache_score(rules, patient_data):
    points = 0.0
    for rule in rules:
        # evaluate all rules
        points += evaluate_rule(rules, rule, patient_data)
    return points

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [118]:
apache_score({ "Organ Failure History" : [
        { "matches": "Nonoperative", "points": 5 },
        { "matches": "Emergency", "points": 5 },
        { "matches": "Elective", "points": 2 },
        { "matches": "None", "points": 0 }
    ], }, {
        "Organ Failure History": "Emergency"
    })

5.0

In [119]:
apache_score({
    "Temperature": [
        { "min": 0, "max": 30, "points": 4 },
        { "min": 30, "max": 32, "points": 3 },
        { "min": 32, "max": 34, "points": 2 },
        { "min": 34, "max": 36, "points": 1 },
        { "min": 36, "max": 38.5, "points": 0 },
        { "min": 38.5, "max": 39, "points": 1 },
        { "min": 39, "max": 41, "points": 3 },
        { "min": 41, "max": 100, "points": 4 }
    ],
    "Heart Rate": [
        { "min": 0, "max": 40, "points": 4 },
        { "min": 40, "max": 55, "points": 3 },
        { "min": 55, "max": 70, "points": 2 },
        { "min": 70, "max": 110, "points": 0 },
        { "min": 110, "max": 140, "points": 2 },
        { "min": 140, "max": 180, "points": 3 },
        { "min": 180, "max": 999, "points": 4 }
    ],
}, {
    "Temperature": 33,
    "Heart Rate": 72
})

2.0

In [120]:
apache_score({
    "FiO2": [
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 0, "max": 55 } ], "points": 4 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 55, "max": 61 } ], "points": 3 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 61, "max": 71 } ], "points": 1 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 71, "max": 999 } ], "points": 0 },
        
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 0, "max": 200 } ], "points": 0},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 200, "max": 350 } ], "points": 2},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 350, "max": 500 } ], "points": 3},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 500, "max": 999 } ], "points": 4}
    ],
    "Glasgow Coma Scale": [
        {"min": 1, "max": 16, "points": "15-_"}
    ]
}, {
    "FiO2": 40,
    "PaO2": 75,
    "Glasgow Coma Scale": 13
})

2.0

In [121]:
assert(apache_score({ "Organ Failure History" : [
        { "matches": "Nonoperative", "points": 5 },
        { "matches": "Emergency", "points": 5 },
        { "matches": "Elective", "points": 2 },
        { "matches": "None", "points": 0 }
    ], }, {
        "Organ Failure History": "Emergency"
    }) == 5)

assert(apache_score({
    "Temperature": [
        { "min": 0, "max": 30, "points": 4 },
        { "min": 30, "max": 32, "points": 3 },
        { "min": 32, "max": 34, "points": 2 },
        { "min": 34, "max": 36, "points": 1 },
        { "min": 36, "max": 38.5, "points": 0 },
        { "min": 38.5, "max": 39, "points": 1 },
        { "min": 39, "max": 41, "points": 3 },
        { "min": 41, "max": 100, "points": 4 }
    ],
    "Heart Rate": [
        { "min": 0, "max": 40, "points": 4 },
        { "min": 40, "max": 55, "points": 3 },
        { "min": 55, "max": 70, "points": 2 },
        { "min": 70, "max": 110, "points": 0 },
        { "min": 110, "max": 140, "points": 2 },
        { "min": 140, "max": 180, "points": 3 },
        { "min": 180, "max": 999, "points": 4 }
    ],
}, {
    "Temperature": 33,
    "Heart Rate": 72
}) == 2)

assert(apache_score({
    "FiO2": [
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 0, "max": 55 } ], "points": 4 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 55, "max": 61 } ], "points": 3 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 61, "max": 71 } ], "points": 1 },
        { "rules": [ { "min": 0, "max": 50 }, { "prop": "PaO2", "min": 71, "max": 999 } ], "points": 0 },
        
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 0, "max": 200 } ], "points": 0},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 200, "max": 350 } ], "points": 2},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 350, "max": 500 } ], "points": 3},
        { "rules": [ { "min": 50, "max": 101}, { "prop": "A-a Gradient", "min": 500, "max": 999 } ], "points": 4}
    ],
    "Glasgow Coma Scale": [
        {"min": 1, "max": 16, "points": "15-_"}
    ]
}, {
    "FiO2": 40,
    "PaO2": 75,
    "Glasgow Coma Scale": 13
}) == 2)

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [122]:
import pandas as pd
import io, requests

patients_data = []
    # read csv file from url into list of dictionaries
    # for example the following csv table -
    #   col1     col2
    #     1       2
    #     3       0
    #
    # will be converted to -
    #    [ { "col1": 1, "col2": 2 }, { "col1": 3, "col2": 0 } ]
url = "https://hds5210-2020.s3.amazonaws.com/TestPatients.csv"
data = pd.read_csv(io.StringIO(requests.get(url).content.decode('utf-8')))
keys = []
for key in data:
    keys.append(key)
row_count = len(data[keys[0]])
for i in range(row_count):
    temp = {}
    for key in keys:
        temp[key] = data[key][i]
    patients_data.append(temp)


for patient_data in patients_data:
    # print apache scores for all patients
    print(apache_score(rules, patient_data))

35.0
31.0
47.0
34.0
44.0
35.0
31.0
49.0
40.0
48.0
42.0
43.0
32.0
41.0
42.0
49.0
37.0
37.0
38.0
43.0
41.0
31.0
38.0
30.0
41.0
41.0
34.0
46.0
40.0
47.0
36.0
43.0
41.0
46.0
44.0
40.0
39.0
37.0
41.0
30.0
46.0
30.0
41.0
44.0
35.0
36.0
40.0
40.0
30.0
50.0
52.0
43.0
46.0
34.0
33.0
42.0
41.0
31.0
46.0
46.0
34.0
36.0
33.0
38.0
26.0
29.0
46.0
25.0
40.0
38.0
39.0
34.0
39.0
53.0
44.0
49.0
37.0
36.0
36.0
51.0
33.0
36.0
43.0
41.0
24.0
50.0
29.0
40.0
36.0
50.0
29.0
37.0
34.0
45.0
34.0
40.0
37.0
37.0
47.0
31.0
33.0
50.0
28.0
37.0
33.0
44.0
40.0
38.0
40.0
31.0
44.0
37.0
44.0
47.0
25.0
34.0
32.0
41.0
38.0
34.0
38.0
39.0
41.0
34.0
29.0
44.0
22.0
40.0
34.0
43.0
40.0
31.0
35.0
33.0
27.0
44.0
52.0
40.0
53.0
40.0
40.0
48.0
31.0
42.0
49.0
43.0
35.0
31.0
36.0
42.0
34.0
31.0
30.0
28.0
38.0
47.0
32.0
47.0
31.0
41.0
40.0
33.0
35.0
42.0
49.0
55.0
51.0
37.0
53.0
29.0
33.0
46.0
36.0
32.0
33.0
33.0
42.0
32.0
41.0
35.0
39.0
40.0
40.0
35.0
28.0
33.0
42.0
29.0
30.0
39.0
34.0
46.0
30.0
37.0
31.0
37.0
41.0
47.0
45.0
29.0


### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv
